# Feature Importance for Genetic Target Validation

*Classifying 5 tumor types based on their mRNA expression levels in 20,532 genes.*

![geneBanner](../images/gene_banner.png)

---

This dataset is comprised of:

* *Features* - the expression of 20,532 genes from 800 participants. 
* *Labels* - the type of tumor: BRCA, KIRC, LUAD or PRAD.

> *Source:* The Cancer Genome Atlas (TCGA)
> https://archive.ics.uci.edu/ml/datasets/gene+expression+cancer+RNA-Seq
>
> Unfortunately, the gene names were not published with the dataset. I contacted the submitting organization, but they were not able to provide an index.

Although the oncogene relationships already known (the tumors are named after gene complexes), the purpose of this experiment is to validate that neural networks can be used to rapidly reveal these patterns in the presence of highly dimensional data. This methodology could also be repurposed for delineating biomarkers of different stages/ subtypes within a single disease.

---

# Preprocessing

Prior to ingestion:

* Balanced the dataset based on the 'Class' column, which essentially meant deleting 150 BRCA samples.

* Filtered out low expression frequency (EF) genes, but that still left over 19.5K genes.

In [2]:
import aiqc
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
splitset = aiqc.Pipeline.Tabular.make(
    # --- Data source ---
    df_or_path = "/Users/layne/desktop/TCGA/TCGA_expression.parquet"

    # --- Label preprocessing ---
    , label_column = 'Class'
    , label_encoder = dict(sklearn_preprocess=OneHotEncoder())

    # --- Feature preprocessing ---
    , feature_cols_excluded = ['Class', 'sample_ID']
    , feature_encoders = [
        dict(sklearn_preprocess=StandardScaler(), dtypes='float64')
    ]

    # --- Stratification ---
    , size_validation = 0.22
    , size_test = 0.08
)

# Modeling

In [200]:
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from keras.callbacks import History

In [201]:
def fn_build(features_shape, label_shape, **hp):
    model = Sequential()
    model.add(Input(shape=features_shape))
    
    # First hidden layer.
    model.add(Dense(hp['first_neurons'], kernel_initializer=hp['init']))
    model.add(BatchNormalization())
    model.add(Activation(hp['activation']))
    model.add(Dropout(hp['drop_rate']))
    
    # Output layer
    model.add(Dense(units=label_shape[0], activation='sigmoid', kernel_initializer='glorot_uniform'))
    return model

In [205]:
def fn_train(model, loser, optimizer, samples_train, samples_evaluate, **hp):
    model.compile(loss=loser, optimizer=optimizer)

    model.fit(
        samples_train['features'], samples_train['labels']
        , validation_data = (samples_evaluate['features'], samples_evaluate['labels'])
        , verbose = 0
        , batch_size = hp['batch_size']
        , epochs = hp['epochs']
        , callbacks = [History()]
    )
    return model

In [202]:
import tensorflow as tf
def fn_optimize(**hp):
    optimizer = tf.keras.optimizers.Adamax(hp['learning_rate'])
    return optimizer

In [203]:
hyperparameters = dict(
    first_neurons   = [120],
    activation      = ['relu'],
    init            = ['he_uniform'],
    epochs          = [10],
    batch_size      = [8],
    drop_rate       = [0.4],
    learning_rate   = [0.01]
)

In [213]:
queue = aiqc.Experiment.make(
    # --- Analysis type ---
    library             = "keras"
    , analysis_type     = "classification_multi"

    # --- Model functions ---
    , fn_build          = fn_build
    , fn_train          = fn_train
    , fn_lose           = None #auto
    , fn_optimize       = fn_optimize #auto
    , fn_predict        = None #auto

    # --- Training options ---
    , repeat_count      = 2
    , permutation_count = 5
    , hyperparameters   = hyperparameters
    , pick_percent      = None

    # --- Data source ---
    , splitset_id       = splitset.id
    , foldset_id        = None
    , hide_test         = False
)

Ironically, the neural network takes less than 30 seconds to train. Whereas the feature permutations take 3+ hours.

In [214]:
queue.run_jobs()

🔮 Training Models 🔮:  50%|█████████████████▌                 | 1/2 [3:10:21<3:10:21, 11421.46s/it]


Queue was gracefully interrupted.



The patterns must be shockingly absolute. After our first try with a single hidden layer, we achieve a perfect score. This is surprisingly given the use of a validation and test split, and given the fact that there are 4 different categories.

In [215]:
queue.metrics_to_pandas()

,hyperparamcombo_id,job_id,repeat_index,predictor_id,split,accuracy,f1,loss,precision,recall,roc_auc
0,240,240,0,671,train,1.0,1.0,0.103631,1.0,1.0,1.0
1,240,240,0,671,validation,1.0,1.0,0.127999,1.0,1.0,1.0
2,240,240,0,671,test,1.0,1.0,0.134419,1.0,1.0,1.0


In [4]:
id = 671
predictor = aiqc.Predictor.get_by_id(id)
prediction = predictor.predictions[0]

In [ ]:
prediction.plot_confusion_matrix()

![geneConfusion](../images/gene_confusion.png)

---

# Interpretation

The `Experiment.permutation_count` parameter determines how many times each feature is permuted and run back through the model. The median difference in loss is then compared to the baseline loss of the model.

In [ ]:
prediction.plot_feature_importance(top_n=30)

![geneFeatures](../images/gene_features.png)

In [ ]:
import pandas as pd
df = pd.read_parquet("/Users/layne/desktop/TCGA/TCGA_expression.parquet")

In [38]:
import plotly.express as px
px.box(df, x="Class", y='gene_15589', height=50).show()
px.box(df, x="Class", y='gene_17801', height=50).show()

![geneBRCA](../images/gene_brca.png)

![geneGroup](../images/gene_group.png)

Interpretting the top 30 features in box plots, we can observe that: 

* BRCA expression is independent from the others. It is significantly more expressed across our top 5 candidate genes. The signal/ patterns are stronger.

* The PRAD, LUAD, and KIRC, tumors appear to be coexpressed. Perhaps those cancers share a pathway.

* There is an accumulation of over-expression across many genes, not just 1 or 2.

* If we had a 5th control group of benign samples, we could learned a lot more.